# Imports and setting up viz

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_routing import *

# Load files
File to evaluate the behaviour of native routing compared to standard.

First working on the same topographic grid (0.5°).

3 files are considered : one from the standard routing simulation, 2 from the native routing simulation (sechiba_history, diag_routing_r)

In [ ]:
dir='../../../JZ_simu_outputs/routing_native'

In [ ]:
spinupyears=[2000,2001,2002]
# spinupyears=[]

## Format functions

In [ ]:
def format_routing_ORC(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds = ds.rename({'time_counter': 'time'})
    ds.attrs['name'] = name
    ds = remove_years_efficiently(ds, spinupyears)

    #edit units and names
    if 'hydrographs' in ds:
        ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
        ds['hydrographs'].attrs['long_name'] = 'River discharge'
    if 'streamr' in ds:
        ds['streamr'].attrs['units'] = 'kg m⁻²'
        ds['streamr'].attrs['long_name'] = 'River reservoir volume'
    if 'slowr' in ds:
        ds['slowr'].attrs['units'] = 'kg m⁻²'
        ds['slowr'].attrs['long_name'] = 'GW reservoir volume'
    if 'fastr' in ds:
        ds['fastr'].attrs['units'] = 'kg m⁻²'
        ds['fastr'].attrs['long_name'] = 'Overland reserv. volume'

    if color is not None:
        ds.attrs['plot_color'] = color

    return ds

In [ ]:
def format_routing_native(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    dict = {'time_counter' : 'time',
        'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_drainage_r' : 'drainage',
        'routing_runoff_r' : 'runoff',
        'routing_hydrographs_r' : 'hydrographs',
        'routing_riverflow_r' : 'riverflow',
        'routing_coastalflow_r' : 'coastalflow'
        }
    ds = ds.rename(dict)
    ds = ds.isel(lon=slice(2,-2),lat=slice(2,-2))

    ds = remove_years_efficiently(ds, spinupyears)

    if color is not None:
        ds.attrs['plot_color'] = color

    #edit units and names
    if 'hydrographs' in ds:
        ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
        ds['hydrographs'].attrs['long_name'] = 'River discharge'
    if 'streamr' in ds:
        ds['streamr'].attrs['units'] = 'kg m⁻²'
        ds['streamr'].attrs['long_name'] = 'River reservoir volume'
    if 'slowr' in ds:
        ds['slowr'].attrs['units'] = 'kg m⁻²'
        ds['slowr'].attrs['long_name'] = 'GW reservoir volume'
    if 'fastr' in ds:
        ds['fastr'].attrs['units'] = 'kg m⁻²'
        ds['fastr'].attrs['long_name'] = 'Overland reserv. volume'
        
    #edit units and names
    ds['hydrographs'].attrs['units'] = 'm³ s⁻¹'
    ds['hydrographs'].attrs['long_name'] = 'River discharge'

    #to move all to 1st day of the month (instead of 14, 15, or 16..)
    ds = ds.resample(time='MS').mean()
    return ds

In [ ]:
def format_routing_nativeDIAG(filename, name, spinupyears, color=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name'] = name

    dict = {'time_counter' : 'time',
            
            # for reservoirs
            'lat_domain_landpoints_routing':'lat',
            'lon_domain_landpoints_routing':'lon',
            # for drainage and runoff
            # 'lat_domain_landpoints':'lat',
            # 'lon_domain_landpoints':'lon',

            'routing_stream_reservoir' : 'streamr',
            'routing_fast_reservoir' : 'fastr',
            'routing_slow_reservoir' : 'slowr',
            'routing_drainage' : 'drainage',
            'routing_runoff' : 'runoff',
            'routing_riverflow' : 'riverflow',
            'routing_coastalflow' : 'coastalflow',
            'routing_irrigation' : 'irrigation',}
    #change the variables that exist according to the dictionary
    ds = ds.rename(dict)
    #remove first years to consider a spinup
    ds = remove_years_efficiently(ds, spinupyears)

    if color is not None:
        ds.attrs['plot_color'] = color

    return(ds)

## No irrigation

### ORC files

In [ ]:
#open netcdf files for standard routing
filename = '{}/eval_halfdeg/nat7.1/MO/nat7.1_*_1M_sechiba_history.nc'.format(dir)
name= 'subgrid_halfdeg'
std_noirr_orig = format_routing_ORC(filename, name, spinupyears, color='dimgrey')
#downsample to monthly values
std_noirr = std_noirr_orig.resample(time='MS').mean()
std_noirr

In [ ]:
filename = '{}/calibration_MERIT/nat12.3/nat12.3*_1M_sechiba_history.nc'.format(dir)
name = 'tcst1'
ORC_tcst1 = format_routing_ORC(filename, name, spinupyears, color='orange')
ORC_tcst1

In [ ]:
filename = '{}/calibration_MERIT/nat12/dt1800_merit_correct_tcst/nat12_*_1M_sechiba_history.nc'.format(dir)
name = 'tcst2'
ORC_tcst2 = format_routing_ORC(filename, name, spinupyears, color='green')
ORC_tcst2

In [ ]:
filename = '{}/calibration_MERIT/nat12.2/correct_tcst/nat12.2*_1M_sechiba_history.nc'.format(dir)
name = 'tcst3'
ORC_tcst3 = format_routing_ORC(filename, name, spinupyears, color='purple')
ORC_tcst3

In [ ]:
filename = '{}/calibration_MERIT/tcst7/tcst7*_1M_sechiba_history.nc'.format(dir)
name = 'tcst4'
ORC_tcst4 = format_routing_ORC(filename, name, spinupyears, color='red')
ORC_tcst4

### DIAG_R files

In [ ]:
filename = '{}/calibration_MERIT/nat12.3/nat12.3*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst1'
DIAGR_tcst1 = format_routing_native(filename, name, spinupyears, color='orange')
DIAGR_tcst1

In [ ]:
filename = '{}/calibration_MERIT/nat12/dt1800_merit_correct_tcst/nat12_*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst2'
DIAGR_tcst2 = format_routing_native(filename, name, spinupyears, color='green')
DIAGR_tcst2

In [ ]:
filename = '{}/calibration_MERIT/nat12.2/correct_tcst/nat12.2*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst3'
DIAGR_tcst3 = format_routing_native(filename, name, spinupyears, color='purple')
DIAGR_tcst3

In [ ]:
filename = '{}/calibration_MERIT/tcst7/tcst7*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4'
name = 'tcst4, no_irr'
DIAGR_tcst4 = format_routing_native(filename, name, spinupyears, color='red')
DIAGR_tcst4

## Sensitivities (irrigation, GSWP3)

In [ ]:
filename = '{}/calibration_MERIT/tcst7irr/nat18*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, irr'
DIAGR_tcst4_irr = format_routing_native(filename, name, spinupyears, color='blue')
DIAGR_tcst4_irr

In [ ]:
filename = '{}/calibration_MERIT/tcst7irr/nat18*_history.nc'.format(dir)
name = 'tcst4, irr'
ORC_tcst4_irr = format_routing_ORC(filename, name, spinupyears, color='blue')

filename = '{}/calibration_MERIT/tcst7irr/nat18*_diag_routing.nc'.format(dir)
DIAG_tcst4_irr = format_routing_nativeDIAG(filename, name, spinupyears, color='blue')

#recompute irrig
ORC_tcst4_irr['irrigation'] = DIAG_tcst4_irr['irrigation'] / ORC_tcst4_irr['Areas'] * 2 * 24

In [ ]:
filename = '{}/calibration_MERIT/tcst7irr0.6/tcst7irr0.6*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, irr_reduced'
DIAGR_tcst4_irr_reduced = format_routing_native(filename, name, spinupyears, color='deepskyblue')
DIAGR_tcst4_irr_reduced

In [ ]:
filename = '{}/calibration_MERIT/tcst7irr0.6/tcst7irr0.6*_history.nc'.format(dir)
name = 'tcst4, irr_reduced'
ORC_tcst4_irr_reduced = format_routing_ORC(filename, name, spinupyears, color='deepskyblue')

filename = '{}/calibration_MERIT/tcst7irr0.6/tcst7irr0.6*_diag_routing.nc'.format(dir)
DIAG_tcst4_irr_reduced = format_routing_nativeDIAG(filename, name, spinupyears, color='deepskyblue')
DIAG_tcst4_irr_reduced

#recompute irrig
ORC_tcst4_irr_reduced['irrigation'] = DIAG_tcst4_irr_reduced['irrigation'] / ORC_tcst4_irr_reduced['Areas'] * 2 * 24

In [ ]:
filename = '{}/calibration_MERIT/tcst7GSWP3/tcst7GSWP3*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, no_irr, GSWP3'
DIAGR_tcst4_gswp3 = format_routing_native(filename, name, spinupyears, color='darkorange')
DIAGR_tcst4_gswp3

In [ ]:
filename = '{}/calibration_MERIT/tcst7irrGSWP3/nat18GSWP3*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, irr, GSWP3'
DIAGR_tcst4_gswp3_irr = format_routing_native(filename, name, spinupyears, color='mediumpurple')
DIAGR_tcst4_gswp3_irr

## Longer sims

In [ ]:
spinupyears_long = [1980, 1981, 1982]

In [ ]:
filename = '{}/calibration_MERIT/tcst7_long/tcst7*_1M_sechiba_history.nc'.format(dir)
name = 'tcst4, no_irr'
ORC_tcst4_long = format_routing_ORC(filename, name, spinupyears_long, color='red')
ORC_tcst4_long

In [ ]:
filename = '{}/calibration_MERIT/tcst7_long/tcst7*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, no_irr'
DIAGR_tcst4_long = format_routing_native(filename, name, spinupyears_long, color='red')
DIAGR_tcst4_long

In [ ]:
#with long sim
filename = '{}/calibration_MERIT/tcst7irr_long/nat18*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, irr'
DIAGR_tcst4_irr_long = format_routing_native(filename, name, spinupyears_long, color='blue')

filename = '{}/calibration_MERIT/tcst7irr_long/nat18*_history.nc'.format(dir)
name = 'tcst4, irr'
ORC_tcst4_irr_long = format_routing_ORC(filename, name, spinupyears_long, color='blue')

filename = '{}/calibration_MERIT/tcst7irr_long/nat18*_diag_routing.nc'.format(dir)
DIAG_tcst4_irr_long = format_routing_nativeDIAG(filename, name, spinupyears_long, color='blue')

#recompute irrig
ORC_tcst4_irr_long['irrigation'] = DIAG_tcst4_irr_long['irrigation'] / ORC_tcst4_irr_long['Areas'] * 2 * 24

In [ ]:
# with long sim
filename = '{}/calibration_MERIT/tcst7irr0.6_long/tcst7irr0.6*_1D_diag_routing_r.nc'.format(dir)
name = 'tcst4, irr_reduced'
DIAGR_tcst4_irr_reduced_long = format_routing_native(filename, name, spinupyears_long, color='deepskyblue')

# filename = '{}/calibration_MERIT/tcst7irr0.6_long/tcst7irr0.6*_history.nc'.format(dir)
# name = 'tcst4, irr_reduced'
# ORC_tcst4_irr_reduced_long = format_routing_ORC(filename, name, spinupyears_long, color='deepskyblue')

# filename = '{}/calibration_MERIT/tcst7irr0.6_long/tcst7irr0.6*_diag_routing.nc'.format(dir)
# DIAG_tcst4_irr_reduced_long = format_routing_nativeDIAG(filename, name, spinupyears_long, color='deepskyblue')

# #recompute irrig
# ORC_tcst4_irr_reduced_long['irrigation'] = DIAG_tcst4_irr_reduced_long['irrigation'] / ORC_tcst4_irr_reduced_long['Areas'] * 2 * 24

## Masks

In [ ]:
#Iberian Peninsula mask
mask_IP = polygon_to_mask(std_noirr, iberian_peninsula)

In [ ]:
ip_std_noirr = std_noirr.where(mask_IP, drop=True)

In [ ]:
ip_ORC_tcst1 = ORC_tcst1.where(mask_IP, drop=True)
# ip_DIAGR_tcst1 = DIAGR_tcst1.where(mask_IP, drop=True)
ip_ORC_tcst2 = ORC_tcst2.where(mask_IP, drop=True)
# ip_DIAGR_tcst2 = DIAGR_tcst2.where(mask_IP, drop=True)
ip_ORC_tcst3 = ORC_tcst3.where(mask_IP, drop=True)
# ip_DIAGR_tcst3 = DIAGR_tcst3.where(mask_IP, drop=True)
ip_ORC_tcst4 = ORC_tcst4.where(mask_IP, drop=True)
# ip_DIAGR_tcst4 = DIAGR_tcst4.where(mask_IP, drop=True)

In [ ]:
ip_ORC_tcst4_irr = ORC_tcst4_irr.where(mask_IP, drop=True)
ip_ORC_tcst4_irr_reduced = ORC_tcst4_irr_reduced.where(mask_IP, drop=True)

In [ ]:

mask_IP_long = polygon_to_mask(ORC_tcst4_long, iberian_peninsula)
ip_ORC_tcst4_long = ORC_tcst4_long.where(mask_IP_long, drop=True)
ip_ORC_tcst4_irr_long = ORC_tcst4_irr_long.where(mask_IP_long, drop=True)
# ip_ORC_tcst4_irr_reduced_long = ORC_tcst4_irr_reduced_long.where(mask_IP_long, drop=True)

## Obs

In [ ]:
proper_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6226400: {'name': 'Zaragoza',           'river': 'Ebro',            'lat_grid': 41.67499,   'lon_grid': -0.90832,   'station_nb': 2},
    6226300: {'name': 'Castejon',           'river': 'Ebro',            'lat_grid': 42.17499,   'lon_grid': -1.69165,   'station_nb': 3},
    6226600: {'name': 'Seros',              'river': 'Segre',           'lat_grid': 41.45833,   'lon_grid': 0.425007,   'station_nb': 4},
    6226650: {'name': 'Fraga',              'river': 'Cinca',           'lat_grid': 41.52499,   'lon_grid': 0.341674,   'station_nb': 5},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6212700: {'name': 'Peral De Arlanza',   'river': 'Arlanza',         'lat_grid': 42.07500,   'lon_grid': -4.07499,   'station_nb': 7},
    6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6213800: {'name': 'Trillo',             'river': 'Tagus',           'lat_grid': 40.70833,   'lon_grid': -2.57499,   'station_nb': 9},
    6213900: {'name': 'Peralejos',          'river': 'Tagus',           'lat_grid': 40.59166,   'lon_grid': -1.92499,   'station_nb': 10},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6116200: {'name': 'Pulo do Lobo',       'river': 'Guadiana',        'lat_grid': 37.803,     'lon_grid': -7.633,     'station_nb': 12},         
    6216530: {'name': 'La Cubeta',          'river': 'Guadiana',        'lat_grid': 38.975,     'lon_grid': -2.895,     'station_nb': 13},         
    6216520: {'name': 'Villarubia',         'river': 'Guadiana',        'lat_grid': 39.125,     'lon_grid': -3.59073,   'station_nb': 14},      
    6216800: {'name': 'Quintanar',          'river': 'Giguela',         'lat_grid': 39.64166,   'lon_grid': -3.07499,   'station_nb': 15},
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
    6217200: {'name': 'Arroyo Maria',       'river': 'Guadalquivir',    'lat_grid': 38.17905,   'lon_grid': -2.83594,   'station_nb': 17}, 
    6217700: {'name': 'Pinos Puente',       'river': 'Frailes',         'lat_grid': 37.27499,   'lon_grid': -3.75832,   'station_nb': 18},
}

#keeping only 5 representative stations for larger rivers, for simple figure
representative_stations_dict = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    # 6213700: {'name': 'Talavera',           'river': 'Tagus',           'lat_grid': 39.95833,   'lon_grid': -4.82499,   'station_nb': 8},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
}

stations_calib_v0 = {
    6298249: {'name': 'Zaragoza',               'river': 'Ebro',        'lon_grid':-0.8749926686286926, 'lat_grid':41.65833282470703,   'year_min':2003, 'year_max':2012, 'station_nb':1},
    6298992: {'name': 'Albarreal De Tajo',      'river': 'Tagus',       'lon_grid':-4.17499303817749,   'lat_grid':39.891666412353516,  'year_min':2003, 'year_max':2012, 'station_nb':2},
    6298481: {'name': 'San Miguel del Pino',    'river': 'Douro',       'lon_grid':-4.92499303817749,   'lat_grid':41.508331298828125,  'year_min':2003, 'year_max':2012, 'station_nb':3},
    6298259: {'name': 'Badajoz',                 'river': 'Guadiana',    'lon_grid': -7.008326530456543, 'lat_grid': 38.85833358764648,  'year_min':2003, 'year_max':2012, 'station_nb':4},
                }

stations_calib_v1 = {
    6226800: {'name': 'Tortosa',            'river': 'Ebro',            'lat_grid': 40.82500,   'lon_grid': 0.525007,   'station_nb': 1},
    6226400: {'name': 'Zaragoza',           'river': 'Ebro',            'lat_grid': 41.67499,   'lon_grid': -0.90832,   'station_nb': 2},
    6226300: {'name': 'Castejon',           'river': 'Ebro',            'lat_grid': 42.17499,   'lon_grid': -1.69165,   'station_nb': 3},
    6226600: {'name': 'Seros',              'river': 'Segre',           'lat_grid': 41.45833,   'lon_grid': 0.425007,   'station_nb': 4},
    6226650: {'name': 'Fraga',              'river': 'Cinca',           'lat_grid': 41.52499,   'lon_grid': 0.341674,   'station_nb': 5},
    6212410: {'name': 'Tore',               'river': 'Douro',           'lat_grid': 41.50833,   'lon_grid': -5.47499,   'station_nb': 6},
    6212700: {'name': 'Peral De Arlanza',   'river': 'Arlanza',         'lat_grid': 42.07500,   'lon_grid': -4.07499,   'station_nb': 7},
    6213800: {'name': 'Trillo',             'river': 'Tagus',           'lat_grid': 40.70833,   'lon_grid': -2.57499,   'station_nb': 9},
    6213900: {'name': 'Peralejos',          'river': 'Tagus',           'lat_grid': 40.59166,   'lon_grid': -1.92499,   'station_nb': 10},
    6216510: {'name': 'Azud de Badajoz',    'river': 'Guadiana',        'lat_grid': 38.86199,   'lon_grid': -7.01,      'station_nb': 11}, 
    6116200: {'name': 'Pulo do Lobo',       'river': 'Guadiana',        'lat_grid': 37.803,     'lon_grid': -7.633,     'station_nb': 12},         
    6216530: {'name': 'La Cubeta',          'river': 'Guadiana',        'lat_grid': 38.975,     'lon_grid': -2.895,     'station_nb': 13},         
    6216800: {'name': 'Quintanar',          'river': 'Giguela',         'lat_grid': 39.64166,   'lon_grid': -3.07499,   'station_nb': 15},
    6217140: {'name': 'Mengibar',           'river': 'Guadalquivir',    'lat_grid': 37.98425,   'lon_grid': -3.79939,   'station_nb': 16},     
    6217200: {'name': 'Arroyo Maria',       'river': 'Guadalquivir',    'lat_grid': 38.17905,   'lon_grid': -2.83594,   'station_nb': 17}, 
}

In [ ]:
filename = '../../../obs/streamflow/GRDC_Monthly_Jan20_v1_ES.nc'
obs_routing_polcher = xr.open_dataset(filename)
rename_dict = {'hydrographs': 'runoff_mean',
               'number': 'id',}
obs_routing_polcher = obs_routing_polcher.rename(rename_dict)
#shift time coordinate to match the sims and grdc obs
obs_routing_polcher['time'] = obs_routing_polcher['time'] - np.timedelta64(14, 'D')
obs_routing_polcher

# Tests and checks

In [ ]:
stop

## Maps

In [ ]:
#Display map of var for ds
var='evap'
ds = ip_ORC_tcst1
# ds = ip_natDIAGR_noirr_month
vmin=0
vmax=600
vmin, vmax = None, None
cmap=wet

# title = 'Annual mean ' + ds[var].long_name + ' ({}) '.format(ds.attrs['name'])
title = var
# clabel= ds[var].name + ' (' + ds[var].units + ')'
clabel=None

map_ave(ds, var, cmap=cmap, vmin=vmin, vmax=vmax, title=title, clabel=clabel)

In [ ]:
# Display map of var for 2 sims
var = ''
# max_val = None
max_val = 0.6
cmap = wet

ds1 = ip_std_irr_month
# ds1 = ip_natDIAGR_noirr_month

# ds2 = ip_natDIAGR_noirr_month
# ds2 = cont_ip_natORC_noirr_month
# ds2 = ip_natDIAG_irr
ds2 = ip_natORC_irr_month

plotvar_std = ds1[var].mean(dim='time')
plotvar_std=plotvar_std.where(plotvar_std>0)
plotvar_nat = ds2[var].mean(dim='time')
plotvar_nat=plotvar_nat.where(plotvar_nat>0)

# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(18, 4), subplot_kw={'projection': ccrs.PlateCarree()})
# fig.suptitle(ds1[var].long_name + ' average over the period (2003-2012, {})'.format(ds1[var].units))

# Plot for std dataset
axs[0].coastlines()
plotvar_std.plot(ax=axs[0], transform=ccrs.PlateCarree(), cmap=cmap, vmax=max_val)
axs[0].set_title(f'{ds1.name}')

# Plot for natORC dataset
axs[1].coastlines()
plotvar_nat.plot(ax=axs[1], transform=ccrs.PlateCarree(), cmap=cmap, vmax=max_val)
axs[1].set_title(f'{ds2.name}')

In [ ]:
#Diff map
var='irrigation'
ds1=ip_natORC_irr_month
ds2=ip_std_irr_month
# ds2=ip_natD_noirr_month

vmin=-0.3
vmax= 0.3
# vmin, vmax = None, None
cmap=emb_neutral

title= ds1[var].attrs['long_name'] + ' difference (' + ds1.attrs['name'] + ' - ' + ds2.attrs['name'] + ')'
clabel= ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'


map_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap, title=title, clabel=clabel)
# map_rel_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=cmap)

In [ ]:
#RMSE map
var='irrigation'
ds1 = ip_std_irr_month
ds2 = ip_natORC_irr_month
map_rmse_ave(ds1, ds2, var)

In [ ]:
#RMSE and rRMSE values for multiple variables
varlist=['streamr', 'fastr', 'slowr']
# varlist=['drainage', 'runoff']
ds1=ip_natORC_noirr
ds2=ip_std_noirr
#compute RMSE between ds1 and ds2
for var in varlist:
    rmse = np.sqrt(((ds1[var]-ds2[var])**2).mean(dim='time'))
    # rel_rmse = rmse/ds2[var].mean(dim='time')
    #output the value of the rmse after spatial averaging over all grid points
    print(var + ' RMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str((rmse.mean(dim=['lon','lat']).values)))
    # print(var + ' rRMSE (' + ds1.name + '-' + ds2.name+ ', spatial and temporal mean) : ' + str((rel_rmse.mean(dim=['lon','lat']).values)))

## Time series

In [ ]:
#Average time series
var='mrsos'
year_min=1980
year_max=2012

ds1 = ip_ORC_tcst4_long
# ds2 = ip_natORC_noirr_month
# ds3 = ip_std_irr_month
# ds4 = ip_natORC_irr_month
# ds_list = [ds1, ds2, ds3, ds4]
# ds_list = [ds3, ds4]
ds_list = [ds1]

time_series_ave(ds_list, var, 
                year_min=year_min, year_max=year_max,
                ds_colors=True
                )

In [ ]:
var='evap'
year_min=2003
year_max=2012

ds1 = ip_std_noirr_month
ds2 = ip_natORC_noirr_month
ds3 = ip_std_irr_month
ds4 = ip_natORC_irr_month
ds_list=[ds1, ds2, ds3, ds4]
ds_list = [ds3, ds4]


seasonal_cycle_ave(ds_list, var, 
                   year_min=year_min, year_max=year_max,
                   ds_colors=True
                   )

# Figures

## Time series / seasonnal cycle of volumes on average over Peninsula

In [ ]:
savefig=True
plot_TS=True
plot_SC=True

varlist=['slowr', 'fastr', 'streamr']
# varlist=['slowr']
year_min=2003
year_max=2012

ds1 = ip_std_noirr
ds2 = ip_ORC_tcst1
ds3 = ip_ORC_tcst2
ds4 = ip_ORC_tcst3
ds5 = ip_ORC_tcst4

ds_list = [ds1, ds2, ds3, ds4, ds5]
# ds_list = [ds1]
title='off'

for var in varlist:
    if plot_TS:
        # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        time_series_ave(ds_list, var, 
                        year_min=year_min, year_max=year_max,
                        # vmin=vmin, vmax=vmax,
                        title=title, ylabel=ylabel,
                        ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_time_series_tcsts.png'.format(var), bbox_inches='tight')

    #Seasonal cycle
    if plot_SC:
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
        seasonal_cycle_ave(ds_list, var,
                            year_min=year_min, year_max=year_max,
                            title=title, ylabel=ylabel,
                            # vmin=vmin, vmax=vmax,
                            ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_seasonal_cycle_tcsts.png'.format(var), bbox_inches='tight')

## Débits sur 4 stations, 4 tcsts

In [ ]:
savefig=True
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(18,8))
axes= axes.flatten()
plot_all_sim=False

ds1 = ip_std_noirr
ds2 = DIAGR_tcst1
ds3 = DIAGR_tcst2
ds4 = DIAGR_tcst3
ds5 = DIAGR_tcst4
ds5.attrs['name'] = 'tcst4'

# ds_list=[ds1]
ds_list = [ds2, ds3, ds4, ds5]
for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    ts_station(obs_ds,axes[i], station_id, polcher_ds=polcher_ds, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012)
    ts_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, polcher_ds=polcher_ds, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, plot_all_sim=plot_all_sim, title_letter=title_letter)
if savefig:
    plt.savefig('figures/chap3/river_discharge/merit_tcst_4stations_TS.png', bbox_inches='tight')

In [ ]:
savefig=True
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(16,8))
axes= axes.flatten()
plot_all_sim=False

ds1 = ip_std_noirr
ds2 = DIAGR_tcst1
ds3 = DIAGR_tcst2
ds4 = DIAGR_tcst3
ds5 = DIAGR_tcst4
# ds_list=[ds1]
ds_list = [ds2, ds3, ds4, ds5]

for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds, plot_all_sim=plot_all_sim, title_letter=title_letter)
if savefig:
    plt.savefig('figures/chap3/river_discharge/merit_tcst_4stations_SC.png', bbox_inches='tight')

## Débits 4 stations, WFDEI vs GSWP3

In [ ]:
savefig=True
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(16,8))
axes= axes.flatten()
plot_all_sim=False

ds1 = DIAGR_tcst4
ds1.attrs['name'] = 'tcst4, no_irr, WFDEI'
ds2 = DIAGR_tcst4_irr
ds2.attrs['name'] = 'tcst4, irr, WFDEI'

ds3 = DIAGR_tcst4_gswp3
ds4 = DIAGR_tcst4_gswp3_irr


# ds_list=[ds1]
ds_list = [ds1, ds2, ds3, ds4]

for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds, plot_all_sim=plot_all_sim, title_letter=title_letter)
if savefig:
    plt.savefig('figures/chap3/river_discharge/merit_forcing_4stations_SC.png', bbox_inches='tight')

## Irr vs irr_reduced

### Débits 4 stations, no_irr, irr, irr_reduced

In [ ]:
savefig=True
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(16,8))
axes= axes.flatten()
plot_all_sim=False

ds1 = DIAGR_tcst4
ds1.attrs['name'] = 'tcst4, no_irr'

ds2 = DIAGR_tcst4_irr
ds3 = DIAGR_tcst4_irr_reduced
# ds_list=[ds1]
ds_list = [ds1, ds2, ds3]

for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=2003, year_max=2012, polcher_ds=polcher_ds, plot_all_sim=plot_all_sim, title_letter=title_letter)
if savefig:
    plt.savefig('figures/chap3/river_discharge/merit_irr_4stations_SC.png', bbox_inches='tight')

### Irrigation in irr and irr_reduced

In [ ]:
savefig=True
#maps
ds1 = ip_ORC_tcst4_irr
ds2 = ip_ORC_tcst4_irr_reduced
var='irrigation'
title='off'
vmin, vmax = 0, 0.4
map_ave(ds1, var, cmap=wet, 
        vmin=vmin, vmax=vmax,
        title=title, clabel='Irrigation (mm d⁻¹)')
if savefig:
    plt.savefig('figures/chap3/maps/{}_ave_tcst4_irr.png'.format(var), bbox_inches='tight')

map_ave(ds2, var, cmap=wet,
        vmin=vmin, vmax=vmax,
        title=title, clabel='Irrigation (mm d⁻¹)')
if savefig:
    plt.savefig('figures/chap3/maps/{}_ave_tcst4_irr_reduced.png'.format(var), bbox_inches='tight')

map_diff_ave(ds1, ds2, var, vmin=-0.4, vmax=0.4, cmap=emb_neutralW,
            title=title,
            clabel='Irrigation (mm d⁻¹)')
if savefig:
    plt.savefig('figures/chap3/maps/{}_diff_tcst4_irr_irrreduced.png'.format(var), bbox_inches='tight')

In [ ]:
savefig=True

plot_TS=True
plot_SC=True

ds1 = ip_ORC_tcst4_irr
ds2 = ip_ORC_tcst4_irr_reduced
#add units to the irrigation variable
ds1['irrigation'].attrs['units'] = 'mm d⁻¹'
ds2['irrigation'].attrs['units'] = 'mm d⁻¹'

ds_list= [ds1, ds2]
year_min=2003
year_max=2012

var='irrigation'
title='off'

if plot_TS:
    # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
    ylabel = 'Irrigation (mm d⁻¹)'
    time_series_ave(ds_list, var, 
                    year_min=year_min, year_max=year_max,
                    # vmin=vmin, vmax=vmax,
                    title=title, ylabel=ylabel,
                    ds_colors=True
                    )
    if savefig:
        plt.savefig('figures/chap3/time_series/{}_time_series_irr_reduced.png'.format(var), bbox_inches='tight')
#Seasonal cycle
if plot_SC:
    # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
    ylabel = 'Irrigation (mm d⁻¹)'
    seasonal_cycle_ave(ds_list, var,
                        year_min=year_min, year_max=year_max,
                        title=title, ylabel=ylabel,
                        # vmin=vmin, vmax=vmax,
                        ds_colors=True
                    )
    if savefig:
        plt.savefig('figures/chap3/time_series/{}_seasonal_cycle_irr_reduced.png'.format(var), bbox_inches='tight')

In [ ]:
savefig=True

plot_TS=True
plot_SC=True

ds1 = ip_ORC_tcst4_irr
ds2 = ip_ORC_tcst4_irr_reduced
#add units to the irrigation variable
ds1['irrigation'].attrs['units'] = 'mm d⁻¹'
ds2['irrigation'].attrs['units'] = 'mm d⁻¹'

ds_list= [ds1, ds2]
year_min=2003
year_max=2012

var='root_deficit'
title='off'

if plot_TS:
    # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
    ylabel = 'Irrigation demand (mm d⁻¹)'
    time_series_ave(ds_list, var, 
                    year_min=year_min, year_max=year_max,
                    # vmin=vmin, vmax=vmax,
                    title=title, ylabel=ylabel,
                    ds_colors=True
                    )
    if savefig:
        plt.savefig('figures/chap3/time_series/{}_time_series_irr_reduced.png'.format(var), bbox_inches='tight')
#Seasonal cycle
if plot_SC:
    # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
    ylabel = 'Irrigation demand (mm d⁻¹)'
    seasonal_cycle_ave(ds_list, var,
                        year_min=year_min, year_max=year_max,
                        title=title, ylabel=ylabel,
                        # vmin=vmin, vmax=vmax,
                        ds_colors=True
                    )
    if savefig:
        plt.savefig('figures/chap3/time_series/{}_seasonal_cycle_irr_reduced.png'.format(var), bbox_inches='tight')

### Impact of irr and irr_reduced on reservoirs

In [ ]:
savefig=False
plot_TS=False
plot_SC=True

varlist=['slowr', 'fastr', 'streamr']
year_min=2003
year_max=2012

ds1 = ip_ORC_tcst4
ds2 = ip_ORC_tcst4_irr
ds3 = ip_ORC_tcst4_irr_reduced

ds_list = [ds1, ds2, ds3]
# ds_list = [ds1]
title='off'

for var in varlist:
    if plot_TS:
        # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        time_series_ave(ds_list, var, 
                        year_min=year_min, year_max=year_max,
                        # vmin=vmin, vmax=vmax,
                        title=title, ylabel=ylabel,
                        ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_time_series_irr_reduced.png'.format(var), bbox_inches='tight')

    #Seasonal cycle
    if plot_SC:
        ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
        # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
        seasonal_cycle_ave(ds_list, var,
                            year_min=year_min, year_max=year_max,
                            title=title, ylabel=ylabel,
                            # vmin=vmin, vmax=vmax,
                            ds_colors=True
                        )
        if savefig:
            plt.savefig('figures/chap3/time_series/{}_seasonal_cycle_irr_reduced.png'.format(var), bbox_inches='tight')

### Diff map of reservoir volumes between irr and irr_reduced

In [ ]:
savefig=False
#maps
ds0 = ip_ORC_tcst4
ds1 = ip_ORC_tcst4_irr
ds2 = ip_ORC_tcst4_irr_reduced
varlist=['slowr', 'fastr','streamr']
title='off'
vmin, vmax = None, None

i=0
for var in varlist:
    clabel = ds0[var].attrs['long_name'] + ' rel. diff. (%)'

    #relative difference of two scenarios against noirr
    # vmin, vmax = -75, 75
    # map_rel_diff_ave(ds1, ds0, var, vmin=vmin, vmax=vmax, cmap=emb_neutralW,
    #             title=title,
    #             clabel=clabel,
    #             )
    # map_rel_diff_ave(ds2, ds0, var, vmin=vmin, vmax=vmax, cmap=emb_neutralW,
    #             title=title,
    #             clabel=clabel,
    #             )
    #relative difference between the two irrigation scenarios
    vmin, vmax = -50, 50
    map_rel_diff_ave(ds2, ds1, var, vmin=vmin, vmax=vmax, cmap=emb_neutralW,
                title=title,
                clabel=clabel,
                )
    map_rel_diff_ave(ds1, ds2, var, vmin=vmin, vmax=vmax, cmap=emb_neutralW,
                title=title,
                clabel=clabel,
                )

    #direct viz of reservoir volumes
#     vmins= [0, 0, 0]
#     vmaxs= [60, 2, 30]  
#     vmin, vmax = vmins[i], vmaxs[i]
#     i += 1
#     clabel = ds0[var].attrs['long_name'] + ' (' + ds0[var].attrs['units'] + ')'
#     map_ave(ds1, var, cmap=blues, vmin=vmin, vmax=vmax,
#             title=title, clabel=clabel)
#     map_ave(ds2, var, cmap=blues, vmin=vmin, vmax=vmax,
#             title=title, clabel=clabel)

## Irr vs irr_reduced, long

### Débits 4 stations, no_irr, irr, irr_reduced long

In [ ]:
savefig=True
stations_dict = stations_calib_v0
obs_ds= obs_routing_polcher
polcher_ds = True
fig, axes= plt.subplots(2,2, figsize=(18,8))
axes= axes.flatten()
plot_all_sim=False

ds1 = DIAGR_tcst4_long
ds1.attrs['name'] = 'tcst4, no_irr'

ds2 = DIAGR_tcst4_irr_long
ds3 = DIAGR_tcst4_irr_reduced_long
# ds_list=[ds1]
ds_list = [ds1, ds2, ds3]

for i, (station_id, station_data) in enumerate(stations_dict.items()):
    title_letter=letters[i]
    sc_station(obs_ds,axes[i], station_id, name=station_data['name'], ylabel='River discharge (m³ s⁻¹)', year_min=1983, year_max=2010, polcher_ds=polcher_ds)
    sc_with_obs(ds_list, obs_ds, axes[i], station_id, station_data, ylabel='River discharge (m³ s⁻¹)', year_min=1983, year_max=2010, polcher_ds=polcher_ds, plot_all_sim=plot_all_sim, title_letter=title_letter)
if savefig:
    plt.savefig('figures/chap3/river_discharge/merit_irr_4stations_SC.png', bbox_inches='tight')

### Irrigation in irr and irr_reduced, long

In [ ]:
#map 
ds1 = ip_ORC_tcst4_irr_long
ds2 = ip_ORC_tcst4_irr_reduced_long
var='irrigation'
title='off'
vmin, vmax = 0, 0.4
map_ave(ds1, var, cmap=wet, 
        vmin=vmin, vmax=vmax,
        title=title, clabel='Irrigation (mm d⁻¹)')

map_ave(ds2, var, cmap=wet,
        vmin=vmin, vmax=vmax,
        title=title, clabel='Irrigation (mm d⁻¹)')

map_diff_ave(ds1, ds2, var, vmin=-0.4, vmax=0.4, cmap=emb_neutralW,
            title=title,
            clabel='Irrigation (mm d⁻¹)')

In [ ]:
savefig=False

plot_TS=True
plot_SC=True

ds1 = ip_ORC_tcst4_irr_long
ds2 = ip_ORC_tcst4_irr_reduced_long
#add units to the irrigation variable
ds1['irrigation'].attrs['units'] = 'mm d⁻¹'
ds2['irrigation'].attrs['units'] = 'mm d⁻¹'

ds_list= [ds1, ds2]
yearmin=1983
yearmax=2010

var='irrigation'
title='off'

if plot_TS:
    # title = '{} (Iberian Peninsula average)'.format(ds1[var].attrs['long_name'])
    ylabel = 'Irrigation (mm d⁻¹)'
    time_series_ave(ds_list, var, 
                    year_min=yearmin, year_max=yearmax,
                    # vmin=vmin, vmax=vmax,
                    title=title, ylabel=ylabel,
                    ds_colors=True
                    )
    if savefig:
        plt.savefig('figures/chap3/time_series/{}_time_series_irr_reduced.png'.format(var), bbox_inches='tight')
#Seasonal cycle
if plot_SC:
    # title = 'Mean seasonal cycle (Iberian Peninsula average, ({}-{})'.format(year_min, year_max)
    ylabel = 'Irrigation (mm d⁻¹)'
    seasonal_cycle_ave(ds_list, var,
                        year_min=yearmin, year_max=yearmax,
                        title=title, ylabel=ylabel,
                        # vmin=vmin, vmax=vmax,
                        ds_colors=True
                    )
    if savefig:
        plt.savefig('figures/chap3/time_series/{}_seasonal_cycle_irr_reduced.png'.format(var), bbox_inches='tight')